6.1 Procesamiento de texto

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
bd['text'] = bd['text'].astype(str)

In [ ]:
# Instanciar el tokenizador de tweets
tokenizer = TweetTokenizer()

# Lista para almacenar todos los tokens
all_tokens = []

# Iterar sobre todas las columnas del DataFrame

for text in bd['text']:
    tokens = tokenizer.tokenize(text)
    all_tokens.extend(tokens)

print("all_tokens_title =",len(all_tokens))

In [ ]:
# analisis de frecuencia en palabras
fdist = nltk.FreqDist(all_tokens)
print('Size BoW=',len(fdist))
topwords = fdist.most_common(20)

In [ ]:
x,y = zip(*topwords)
plt.figure(figsize=(15,10))
plt.bar(x,y)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# stopwords en nltk
from nltk.corpus import stopwords

stop_words_nltk = set(stopwords.words('spanish'))

In [ ]:
import re
#  TOKENIZAR con nltk,
# ELIMINAR tokens de long = 1
# ELIMINAR caracteres que no sean alfanumericos
# REMOVER stop words
# graficar los 20 términos más frecuentes:

# ya tokenizado en all_tokens
tokens = [w.lower() for w in all_tokens if len(w)>1]
tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
tokens = [w for w in tokens if w not in stop_words_nltk]

fdist = nltk.FreqDist(tokens)
topwords = fdist.most_common(20)
print('Size of new BoW=',len(fdist))
x,y = zip(*topwords)
plt.figure(figsize=(15,10))
plt.bar(x,y)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Stemming con NLTK

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()
lancaster = LancasterStemmer()


tokens = [lancaster.stem(w) for w in tokens]

fdist = nltk.FreqDist(tokens)
topwords = fdist.most_common(20)
print('Size of new BoW =',len(fdist))


In [ ]:
# Lemmatization con NLTK
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

tokens = [wordnet_lemmatizer.lemmatize(w) for w in tokens ]

fdist = nltk.FreqDist(tokens)
topwords = fdist.most_common(20)
print('Size of new BoW =',len(fdist))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer()

# Ajustar y transformar los documentos en una matriz TF-IDF
tfidf_matrix = vectorizer.fit_transform(tokens)

# Convertir las columnas de la matriz dispersa en un DataFrame disperso
df_tfidf_sparse = pd.DataFrame.sparse.from_spmatrix(tfidf_matrix, columns=vectorizer.get_feature_names_out())




# Mostrar el DataFr

print(df_tfidf_sparse)

In [ ]:
stop_words = set(stopwords.words('spanish'))

# Función de limpieza de texto
def limpiar_texto(text):
    text = re.sub(r'http\S+', '', text)  # Eliminar URLs
    text = re.sub(r'@\w+', '', text)     # Eliminar menciones
    text = re.sub(r'#\w+', '', text)     # Eliminar hashtags
    text = re.sub(r'\d+', '', text)      # Eliminar números
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuaciones
    text = text.lower()                  # Convertir a minúsculas
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar la limpieza a la columna de tuits
bd['texto_limpio'] = bd['text'].apply(limpiar_texto)

In [ ]:
bd['texto_limpio']

In [ ]:
from textblob import TextBlob

# Función para obtener el sentimiento
def obtener_sentimiento(text):
    analisis = TextBlob(text)
    if analisis.sentiment.polarity > 0:
        return 'positivo'
    elif analisis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negativo'

# Aplicar el análisis de sentimiento en la columna limpia
bd['sentimento'] = bd['texto_limpio'].apply(obtener_sentimiento)

In [ ]:
bd['sentimento'].value_counts()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Convertir el texto en una matriz de cuentas (CountVectorizer)
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = count_vectorizer.fit_transform(bd['texto_limpio'])

# Aplicar LDA para identificar los tópicos
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Mostrar los términos principales de cada tópico
def mostrar_topicos(modelo, count_vectorizer, n_palabras):
    palabras = count_vectorizer.get_feature_names_out()
    for idx, topico in enumerate(modelo.components_):
        print(f"Tópico {idx}:")
        print([palabras[i] for i in topico.argsort()[-n_palabras:]])

mostrar_topicos(lda, count_vectorizer, 10)

In [ ]:
pip install pipeline

In [ ]:
from transformers import pipeline

# Cargar el pipeline de análisis de emociones
clasificador = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base')

# Analizar emociones en textos
bd['emociones'] = bd['text'].apply(lambda x: clasificador(x))

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el modelo SBERT
modelo = SentenceTransformer("all-MiniLM-L6-v2")

# Dataset de texto
texto = ["Estoy muy feliz", "Esto es muy frustrante", "Me siento triste"]

# Definir emociones base
emociones_base = {
    "alegría": "Esto me hace muy feliz y emocionado.",
    "tristeza": "Siento una profunda tristeza y desolación.",
    "enojo": "Estoy molesto y frustrado."
}

# Generar embeddings para textos y emociones base
emb_textos = modelo.encode(texto)
emb_emociones = modelo.encode(list(emociones_base.values()))

# Calcular similitud
for i, t in enumerate(texto):
    similitudes = cosine_similarity([emb_textos[i]], emb_emociones)
    emocion_predicha = list(emociones_base.keys())[np.argmax(similitudes)]
    print(f"Texto: {t}\nEmoción: {emocion_predicha}\n")

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el modelo SBERT
modelo = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
texto = bd['text'].tolist()

In [ ]:
# Definir emociones base
emociones_base = {
    "alegría": "Esto me hace muy feliz y emocionado.",
    "tristeza": "Siento una profunda tristeza y desolación.",
    "enojo": "Estoy molesto y frustrado."
}

# Generar embeddings para textos y emociones base
emb_textos = modelo.encode(texto)
emb_emociones = modelo.encode(list(emociones_base.values()))

# Calcular similitud
for i, t in enumerate(texto):
    similitudes = cosine_similarity([emb_textos[i]], emb_emociones)
    emocion_predicha = list(emociones_base.keys())[np.argmax(similitudes)]
    print(f"Texto: {t}\nEmoción: {emocion_predicha}\n")

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pandas as pd

# Cargar el modelo LaBSE
modelo = SentenceTransformer("sentence-transformers/LaBSE")

# Generar vectores
vectores = modelo.encode(bd['text'].tolist())

# Agrupar con KMeans
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(vectores)

# Añadir resultados al DataFrame
bd["cluster"] = clusters

In [ ]:
bd.head(10)

In [ ]:
bd['cluster'].value_counts()

In [ ]:
# Función de limpieza de texto
def limpiar_texto(text):
    text = re.sub(r'http\S+', '', text)  # Eliminar URLs
    text = re.sub(r'@\w+', '', text)     # Eliminar menciones
    text = re.sub(r'#\w+', '', text)     # Eliminar hashtags
    text = re.sub(r'\d+', '', text)      # Eliminar números
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuaciones
    text = text.lower()                  # Convertir a minúsculas
    return text

# Aplicar la limpieza a la columna de tuits
bd['texto_limpio2'] = bd['text'].apply(limpiar_texto)

In [ ]:
texto = bd['texto_limpio2'].tolist()

Modelo preentrenado SBERT

In [ ]:
# Definir emociones base
emociones_base = {
    "positivo": ['Que estilo', 'La tienda más cool del mundo', 'Me merecen esos jogger',  'lo hacen muy bien', 'bonita la ropa', 'me encanta su contenido', 'La mejor ropa','Belleza', 'Wow 😍❤️', 'Me encanta el 3','Últimamente me gusta sus prendas ','Eso si es verdad, excelente calidad en las telas, amo las camisas blancas de ustedes, tengo 3 ','La calidad de las telas si ha mejorado muchísimo. 😍😍😍😍😍','Amo ese vestido blanco lo veo y lo quiero 😍'],
    "neutro": ['precio por favor',  'el top está en la tienda','item de la camiseta', 'lo tienen en laureles medellín talla s' , 'por el sitio web se pueden comprar?',
               'donde están ubicados', 'que item tiene el pantalon'],
    "negativo": ['Hagan tallas más pequeñas',  'la otra vez  no habían', 'Jamás los dejan entrar', 'Traten de que hagan talla grandes', 'la ropa esta cara',
                  'la pagina no deja comprar', 'la ropa esta fea','Buenas tardes realice un pedido y me lo enviaron por TCC pero mirando la guía dice que llevan 2 intentos y no encuentran mi dirección',
                  'Cuando algo que no sea oversize ? 😢','Se les acabaron las ideas? 🥴','saquen otra vez vestiditos sueltos como los de hace años ,hacen falta 😢',
                  'Tuve un problema con una compra me gustaría resolverlo',
                  'Creo que sus camisetas y sus últimas publicaciones acerca de la "friendzone" atentan terriblemente contra el derecho de decir NO de una mujer']

}

# Generar embeddings para emociones base
emociones_textos = [item for sublist in emociones_base.values() for item in sublist]  # Aplanar la lista de emociones
emb_emociones = modelo.encode(emociones_textos)

# Crear un diccionario de emociones con índices
emocion_indices = list(emociones_base.keys())
emocion_dict = {}
index = 0
for emocion, frases in emociones_base.items():
    emocion_dict[emocion] = list(range(index, index + len(frases)))
    index += len(frases)

def predecir_emocion(texto):
    try:
        emb_texto = modelo.encode([texto])
        similitudes = cosine_similarity(emb_texto, emb_emociones)
        max_index = np.argmax(similitudes)  # Obtener el índice de la máxima similitud
        emocion_predicha = None
        for emocion, indices in emocion_dict.items():
            if max_index in indices:
                emocion_predicha = emocion
                break
        return emocion_predicha
    except Exception as e:
        print(f"Error procesando el texto: {texto}, {e}")
        return None

# Aplicar al dataframe
bd['emocion'] = bd['texto_limpio2'].apply(predecir_emocion)

In [ ]:
bd['emocion'].value_counts()

In [ ]:
bd[bd['emocion'] == 'negativo'].tail(10)

In [ ]:
# Cargar el modelo LaBSE
modelo = SentenceTransformer("sentence-transformers/LaBSE")

# Generar vectores
vectores = modelo.encode(bd['texto_limpio2'].tolist())

# Agrupar con KMeans
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(vectores)

# Añadir resultados al DataFrame
bd["cluster"] = clusters

In [ ]:
bd[bd['cluster'] == 0].tail(10)

In [ ]:
# Función de limpieza que conserva emojis
def limpiar_texto_con_emojis(text):
    text = re.sub(r'http\S+', '', text)  # Eliminar URLs
    text = re.sub(r'@\w+', '', text)     # Eliminar menciones
    text = re.sub(r'#\w+', '', text)     # Eliminar hashtags
    text = re.sub(r'\d+', '', text)      # Eliminar números
    text = re.sub(r'[^\w\s\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF]', '', text)
    text = text.lower()                  # Convertir a minúsculas
    return text

bd['texto_limpio3'] = bd['text'].apply(limpiar_texto)

In [ ]:
# Generar vectores
vectores = modelo.encode(bd['texto_limpio3'].tolist())

# Agrupar con KMeans
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(vectores)

# Añadir resultados al DataFrame
bd["cluster"] = clusters

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar un modelo compatible
modelo_beto_sbert = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

print("Modelo BETO-SBERT alternativo cargado con éxito.")